# Inference Test

In [ ]:
import pandas as pd
import time
import sys
import os

# Add project root to path
sys.path.append(os.path.abspath(os.path.join('..')))

from src import config
from src.inference import make_prediction, load_model

print("✅ Libraries loaded. Ready to test inference.")

In [ ]:
# Load the clean data just to get a sample row
df = pd.read_csv(f"../{config.PROCESSED_DATA_PATH}")

# Pick a random row (e.g., row #100)
sample_row = df.iloc[100]
print("--- Sample Input Data ---")
print(sample_row)

# Separate Target (We don't send the answer to the model!)
ground_truth = sample_row[config.TARGET_COLUMN]
input_data = sample_row.drop(config.TARGET_COLUMN).to_dict()

print("\n--- Input Payload (JSON-like) ---")
print(input_data)

In [ ]:
print("⏳ Calling make_prediction()...")

# START TIMER
start_time = time.time()

# CALL THE FUNCTION
result = make_prediction(input_data)

# STOP TIMER
end_time = time.time()
latency = (end_time - start_time) * 1000 # Convert to ms

print(f"\n✅ Prediction Result: {result}")
print(f"⏱️ Latency: {latency:.2f} ms")
print(f"🎯 Actual Value (Ground Truth): {ground_truth}")

In [ ]:
print("running batch stress test...")
success_count = 0
errors = []

# Take 100 random samples
batch = df.sample(100).drop(columns=[config.TARGET_COLUMN]).to_dict(orient='records')

for i, record in enumerate(batch):
    try:
        res = make_prediction(record)
        if res['status'] == 'Success':
            success_count += 1
    except Exception as e:
        errors.append(e)

print(f"Batch Test Complete: {success_count}/100 successful.")
if errors:
    print(f"First Error: {errors[0]}")